# GR5241 Project4 
##  L1 penalized logistic regression + Doubly Robust Estimation 

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import time
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression


###  Low Dimensional data
#### Data processing 
#### 1.1 Create X and Y variables for L1 Logistic Regression 

In [2]:
lowDim = pd.read_csv(r'../data/lowDim_dataset.csv')
X_low = lowDim.drop(['A','Y'], axis = 1)  
y_low = lowDim[['A']]

#### 1.2 Split data into training and testing sets

In [3]:
X_train_low, X_test_low, y_train_low, y_test_low = train_test_split(X_low, y_low, test_size=0.25, random_state=0)

#### 1.3 Standardize features
- Because the regularization penalty is comprised of the sum of the absolute value of the coefficients, we need to scale the data so the coefficients are all based on the same scale.

In [4]:
# Create a scaler object
sc = StandardScaler()
X_train_low_std = sc.fit_transform(X_train_low)
X_test_low_std = sc.transform(X_test_low)

#### 2. L1 penalized logistic regression 
#### 2.1 Parameter tunning 

In [5]:
C = [1, 0.95, 0.9, 0.85, 0.8, 0.75, 0.7, 0.65, 0.5, 0.3, 0.2, 0.1]
for c in C:
    clf = LogisticRegression(penalty='l1', C = c, solver = 'liblinear')
    clf.fit(X_train_low, y_train_low)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_low_std, y_train_low))
    print('Test accuracy:', clf.score(X_test_low_std, y_test_low))
    print('')   

C: 1
Coefficient of each feature: [[ 0.34419367  0.43238251  0.73102716 -0.15445403  0.          0.13306032
   0.          0.26229326  0.          0.33354899 -0.0579125   0.
   0.04015877 -0.50944388  0.14782478  0.          0.24494353  0.10429014
   0.          0.          0.00275068  0.41842302]]
Training accuracy: 0.7837078651685393
Test accuracy: 0.7226890756302521

C: 0.95
Coefficient of each feature: [[ 3.41394331e-01  4.19164596e-01  7.21203160e-01 -1.47436611e-01
   0.00000000e+00  1.30737853e-01  0.00000000e+00  2.60321521e-01
   0.00000000e+00  3.21937086e-01 -4.60120791e-02  0.00000000e+00
   3.64427152e-02 -5.04450999e-01  1.46549264e-01  0.00000000e+00
   2.44899467e-01  1.03654487e-01  0.00000000e+00  0.00000000e+00
   2.75226924e-04  4.16449548e-01]]
Training accuracy: 0.7837078651685393
Test accuracy: 0.7142857142857143

C: 0.9
Coefficient of each feature: [[ 0.33881124  0.40536357  0.710973   -0.13966154  0.          0.12831076
   0.          0.25802403  0.          0.

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A colu

In [37]:
start_time_l1_low = time.time()
# Best: C = 1
clf_low = LogisticRegression(penalty='l1', C = 1, solver = 'liblinear')

#### 2.2 Calculate propensity scores

In [7]:
clf_low.fit(X_low, y_low.values.ravel())
ps_low=clf_low.predict_proba(X_low)[:, 1]

#### 3. Doubly Robust Estimation for ATE

In [8]:
full_low_dim = lowDim.copy()
full_low_dim['PS']=pd.Series(ps_low, index=full_low_dim.index)

In [9]:
# deviding the low dimensional data into treated and control groups

lowDim_treated = lowDim[lowDim['A'] == 1]
lowDim_treated = lowDim_treated.reset_index(drop = True)

lowDim_control = lowDim[lowDim['A'] == 0]
lowDim_control = lowDim_control.reset_index(drop = True)

#Fit a regression model to get the estimation of y given T=1 and X 
X1_low_treated = lowDim_treated.drop(['Y'], axis = 1)
y_low_treated = lowDim_treated['Y']
lr_low_treated = LinearRegression().fit(X1_low_treated, y_low_treated)

# Fit a regression model to get the estimation of y given T=0and X 
X1_low_control = lowDim_control.drop(['Y'], axis = 1)
y_low_control = lowDim_control['Y']
lr_low_control = LinearRegression().fit(X1_low_control, y_low_control)

In [10]:
# Select all covariates and 'A' columns from full dataset
X_low_new = full_low_dim.drop(['Y','PS'], axis = 1)
m1_low= lr_low_treated.predict(X_low_new)
m0_low= lr_low_control.predict(X_low_new)
# join m1 and m0 to full_low_dim
full_low_dim['m1'] = pd.Series(m1_low, index = full_low_dim.index)
full_low_dim['m0'] = pd.Series(m0_low, index = full_low_dim.index)

In [29]:
def DRE(full_data):
    
    n = len(full_data.index)
    result1 = 0
    result2 = 0
    
    for i in range(n):
        result1 = result1 + (full_data['A'][i] * full_data['Y'][i] - (full_data['A'][i] - full_data['PS'][i])*full_data['m1'][i])/full_data['PS'][i]
        result2 = result2 + ((1-full_data['A'][i])* full_data['Y'][i] - (full_data['A'][i] - full_data['PS'][i])*full_data['m0'][i])/(1-full_data['PS'][i])
        
    ETA = 1/n*(result1-result2)
    
    return ETA

In [34]:
start_time_dre_low = time.time()
DRE(full_low_dim)

2.404204395264653

In [46]:
print(f'The estimated ATE for low dimensional data is: {DRE(full_low_dim)}')
low_acc = abs(DRE(full_low_dim) - 2.5)/2.5 * 100
print(f'Accuracy of ATE estimation of low dimensional data is: {low_acc}')
l1_time_low = time.time() - start_time_l1_low 
print(f'Running time of L1 Logist Regression model on low dimensional data is: {l1_time_low} s')
dre_time_low = time.time() - start_time_dre_low
print(f'Running time of DRE model on low dimensional data is: {dre_time_low} s')


The estimated ATE for low dimensional data is: 2.404204395264653
Accuracy of ATE estimation of low dimensional data is: 3.8318241894138794
Running time of L1 Logist Regression model on low dimensional data is: 385.05313205718994 s
Running time of DRE model on low dimensional data is: 408.0132300853729 s


###  High Dimensional data
#### 1. Data processing 
#### 1.1 Create X and Y variables for L1 Logistic Regression 


In [16]:
highDim = pd.read_csv(r'../data/highDim_dataset.csv')
X_high = highDim.drop(['A','Y'], axis = 1)
y_high = highDim[['A']]


#### 1.2 Split data into training and testing sets

In [17]:
X_train_high, X_test_high, y_train_high, y_test_high = train_test_split(X_high, y_high, test_size=0.25, random_state=0)

#### 1.3 Standardize features

In [18]:
X_train_high_std = sc.fit_transform(X_train_high)
X_test_high_std = sc.transform(X_test_high)

#### 2. L1 penalized logistic regression 
#### 2.1 Parameter tunning 

In [19]:
C = [.06, .05, .04, .03, .02, .01, 0.008, 0.005, 0.001]
for c in C:
    clf = LogisticRegression(penalty='l1', C = c, solver = 'liblinear')
    clf.fit(X_train_high, y_train_high)
    print('C:', c)
    print('Coefficient of each feature:', clf.coef_)
    print('Training accuracy:', clf.score(X_train_high_std, y_train_high))
    print('Test accuracy:', clf.score(X_test_high_std, y_test_high))
    print('')



/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


C: 0.06
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.58998854e-02  7.23769086e-03
   1.62599011e-02 -2.35053723e-02  1.71587305e-03  0.00000000e+00
   0.00000000e+00 -2.97405672e-02 -2.15553686e-02  2.76897970e-03
   6.24692230e-03  6.26328618e-03  7.34428317e-03 -2.82796530e-03
  -9.20471166e-05  0.00000000e+00  5.53036570e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.42541770e-02  0.00000000e+00  1.16311084e-02
   8.36507452e-04  0.00000000e+00  3.47943638e-04 -5.16593493e-03
   1.51992704e-05  0.00000000e+00  1.18289213e-02  9.95651111e-03
  -6.77358096e-04 -1.10527225e-03  0.00000000e+00  3.58884161e-05
  -8.65896264e-03  2.31220853e-02 -3.53815315e-02  5.11373606e-02
   5.34869389e-02  0.00000000e+00  0.00000000e+00 -1.74669007e-02
   0.00000000e+00  1.61680691e-02 -5.36437751e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00 -2.40492886e-03  0.00000000e+00
  -1.41986192e-03 -6.12455601e-02  0.00

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


C: 0.05
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.18045120e-02  7.65256882e-03
   1.52302588e-02 -2.17855386e-02  1.50073849e-03  0.00000000e+00
   0.00000000e+00 -2.84674811e-02 -1.32132008e-02  2.39757515e-03
   5.80380925e-03  5.90705681e-03  6.85244169e-03 -2.76855172e-03
  -2.28623858e-04  0.00000000e+00  2.16600147e-03  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.10507163e-02  0.00000000e+00  9.14646341e-03
   8.31477382e-04  0.00000000e+00  3.65764757e-04 -4.80476242e-03
   1.43047335e-05  0.00000000e+00  1.10408262e-02  8.93309453e-03
  -6.46076032e-04 -1.10664790e-03  0.00000000e+00  1.82147529e-05
  -7.31404085e-03  1.92710041e-02 -3.27232850e-02  4.98215262e-02
   4.79009362e-02  0.00000000e+00  0.00000000e+00 -1.40950830e-02
   0.00000000e+00  1.51795098e-02  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -6.00832801e-02  0.00

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


C: 0.03
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  1.68940428e-03  8.79116834e-03
   1.09140648e-02 -1.63896424e-02  7.79454360e-04  0.00000000e+00
   0.00000000e+00 -2.40263544e-02  0.00000000e+00  1.79459587e-03
   4.85049034e-03  5.21183797e-03  5.49701566e-03 -1.99970308e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -2.17162746e-02  0.00000000e+00  3.67672512e-03
   8.82528398e-04  0.00000000e+00  4.80814797e-04 -3.92321616e-03
   1.24851243e-05  0.00000000e+00  7.67335950e-03  3.57075523e-03
  -5.26988265e-04 -1.04568704e-03  0.00000000e+00  1.97127304e-05
  -1.64957150e-03  3.03970612e-03 -1.64938937e-02  4.56144205e-02
   4.07377499e-02  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  8.82866917e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -5.21622446e-02  0.00

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


C: 0.01
Coefficient of each feature: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  4.83240528e-03
   0.00000000e+00  0.00000000e+00  3.44441850e-04  0.00000000e+00
   0.00000000e+00 -7.14826808e-03  0.00000000e+00  0.00000000e+00
   3.74189023e-04  2.04889270e-03  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   9.23339777e-04  0.00000000e+00  2.63763501e-04 -1.70413059e-03
   6.39455810e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  -2.33643890e-04 -7.86559132e-04  0.00000000e+00  1.75814346e-05
   0.00000000e+00  0.00000000e+00  0.00000000e+00  6.04989762e-03
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -1.10026790e-02  0.00

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/utils/validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


#### 2.2 Best model 

In [39]:
# Best: C = 0.06
start_time_l1_high = time.time()
clf_high = LogisticRegression(penalty='l1', C = 0.06, solver = 'liblinear')

#### 2.3 Calculate propensity scores

In [40]:
clf_high.fit(X_high, y_high.values.ravel())
ps_high=clf_high.predict_proba(X_high)[:, 1]

#### 3. Doubly Robust Estimation Algorithm to calculate ATE

In [41]:
full_high_dim= highDim.copy()
full_high_dim['PS']=pd.Series(ps_high, index=full_high_dim.index)

In [42]:
# deviding the high dimensional data into treated and control groups
highDim_treated = highDim[highDim['A'] == 1]
highDim_treated = highDim_treated.reset_index(drop = True)

highDim_control = highDim[highDim['A'] == 0]
highDim_control = highDim_control.reset_index(drop = True)


# Fit a regression model to get the estimation of y given T=1 and X 
X1_high_treated = highDim_treated.drop(['Y'], axis = 1)
y_high_treated = highDim_treated['Y']
lr_high_treated = LinearRegression().fit(X1_high_treated, y_high_treated)


# Fit a regression model to get the estimation of y given T=0 and X 
X1_high_control = highDim_control.drop(['Y'], axis = 1)
y_high_control = highDim_control['Y']
lr_high_control = LinearRegression().fit(X1_high_control, y_high_control)

In [43]:
X_high_new = full_high_dim.drop(['Y','PS'], axis = 1)
m1_high= lr_high_treated.predict(X_high_new)
m0_high= lr_high_control.predict(X_high_new)
full_high_dim['m1'] = pd.Series(m1_high, index = full_high_dim.index)
full_high_dim['m0'] = pd.Series(m0_high, index = full_high_dim.index)

In [44]:
start_time_dre_high = time.time()
DRE(full_high_dim)

-3.0962049953946535

In [47]:
print(f'The estimated ATE for low dimensional data is: {DRE(full_high_dim)}')
high_acc = (DRE(full_high_dim) - (-3))/(-3) * 100
print(f'Accuracy of ATE estimation of low dimensional data is: {high_acc}')
l1_time_high = time.time() - start_time_l1_high 
print(f'Running time of L1 Logist Regression model on low dimensional data is: {l1_time_high} s')
dre_time_high = time.time() - start_time_dre_high
print(f'Running time of DRE model on low dimensional data is: {dre_time_high} s')


The estimated ATE for low dimensional data is: -3.0962049953946535
Accuracy of ATE estimation of low dimensional data is: 3.206833179821785
Running time of L1 Logist Regression model on low dimensional data is: 208.83839631080627 s
Running time of DRE model on low dimensional data is: 201.6145739555359 s


### Summary Table 

In [50]:
summary_table = pd.DataFrame(np.array([[l1_time_low, dre_time_low,low_acc], [l1_time_high, dre_time_high,high_acc]]),
                   columns=['l1 running time', 'DRE running time', 'Accuracy of ATE'],
                            index=['low dim','high dim'])


In [51]:
summary_table

,l1 running time,DRE running time,Accuracy of ATE
low dim,385.053132,408.013230,3.831824
high dim,208.838396,201.614574,3.206833
